In [79]:
import tensorflow as tf
import numpy as np
import pprint
tf.reset_default_graph()

## One node: 4 (input-dim) in 2 (hidden_size)

x데이터를 넣고 어떤 결과가 나온느지 살펴보자

문자를 벡터를 표현하는 가장 좋은 방법은 One hot encoding이다

In [80]:
h=[1,0,0,0]
e=[0,1,0,0]
l=[0,0,1,0]
o=[0,0,0,1]

x_data=np.array([[[1,0,0,0]]],dtype=np.float32)

print(x_data.shape)

(1, 1, 4)


RNN이 기존 뉴럴넷과 다른 점은 RNN의 아웃풋이 다른 cell과 이어진다.

이때 cell에서 나가는 아웃풋(출력)의 크기인 hidden size또는 num_unit을 정해준다. 

이때 크기는 우리 마음대로 정해줄 수 있다.

In [81]:
hidden_size = 2
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

cell 만든것을 구동시켜 입력을 주고 출력값을 뽑아냄

그러기 위해 드라이브 (dynamic 함수)를 이용해 우리가 만든 cell과 데이터(x_data)를 입력시켜준다.

그러면 드라이브 함수는 output과 마지막 state의 값을 출력한다.

In [82]:
outputs, _states= tf.nn.dynamic_rnn(cell,x_data, dtype=tf.float32)

cell을 생성시킬때 우리가 원하는 셀을 만들 수 있음

cell = tf.contrib.rnn.BasicLSTMcell(num_units=hiddent_size)

rnn이 어떤 output을 출력하는지 확인해 보자

In [83]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

아웃풋을 evaluation 하는 방법으로 출력을 해본다.

In [84]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(outputs.eval())

array([[[-0.15228143, -0.54341471]]], dtype=float32)


hidden_size를 2개로 잡아서 2개 나왔다.

나온 값은 원래 학습해야하지만 초기에는 랜덤하게 주어지기 때문에 저렇게 나온 것이다.

## Unfolding to n sequences

sequences 데이터는 3D 데이터를 몇개를 받을 것인가? = 즉, cell을 몇번 펴칠것인가? = sequence_length

입력데이터의 모양에 따라 결정된다.

마찬가지로 hidden_size는 2개로 하자

In [85]:
hidden_size=2
cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
x_data = np.array([[h,e,l,l,o]],dtype=np.float32)
print(x_data.shape)
pp.pprint(x_data)

(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)


여기서 5가 sequence_length이고 cell을 5번 펼친다는 (5번 돈다)라는 것이다.

x_data를 5개 받기 때문이다.

rnn의 cell이 어떤 결과를 내는지 확인해보자.

In [86]:
outputs, _states=tf.nn.dynamic_rnn(cell,x_data, dtype=tf.float32)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
print(outputs.shape)
pp.pprint(outputs.eval())

(1, 5, 2)
array([[[ 0.18433517, -0.10750349],
        [-0.03479795, -0.04428365],
        [ 0.00871665, -0.05401073],
        [ 0.03309646, -0.07108718],
        [ 0.02116945, -0.01878442]]], dtype=float32)


hidden_size=2, sequence_length는 5여서 (1,5,2)의 모양을 출력한다.

## Batching input

학습을 시킬때 몇개의 문자열을 주는가 = batch size


In [91]:
tf.reset_default_graph()

이것은 위에서 이미 만들어놓은 그래프가 아래 그래프를 실행하는데 영향을 줄 수 있다,

따라서 텐서플로우 그래프를 모두 초기화 시켜주는 것이다.

In [92]:
x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

이렇게 되면 hidden_size=2, 각 각 다섯 글자 이므로 sequence_lengh=5, 3개의 문자열이므로 batch_size는 3이 된다.

In [93]:
pp.pprint(x_data)
print(x_data.shape)

cell= tf.contrib.rnn.BasicLSTMCell(num_units=2, state_is_tuple=True)

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
(3, 5, 4)


예상대로 3,5,4 가 출력됐다.

이제 rnn이 어떤 값을 출력하는지 확인해 보자

In [95]:
outputs, _states=tf.nn.dynamic_rnn(cell,x_data,dtype=tf.float32)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
print(outputs.shape)
pp.pprint(outputs.eval())

(3, 5, 2)
array([[[-0.0268018 , -0.0108459 ],
        [-0.14019839,  0.08100206],
        [ 0.0161913 ,  0.12757392],
        [ 0.15328567,  0.16453341],
        [ 0.14175221,  0.0649204 ]],

       [[-0.11882194,  0.0909019 ],
        [-0.09527053, -0.02603546],
        [ 0.05221596,  0.06715646],
        [ 0.17462224,  0.12854126],
        [ 0.26567948,  0.16912252]],

       [[ 0.13037582,  0.08598506],
        [ 0.2315612 ,  0.14219324],
        [ 0.02734536,  0.1828312 ],
        [-0.11041081,  0.20763998],
        [ 0.05363417,  0.20751837]]], dtype=float32)


모양은 예상한대로 3,5,2의 모양으로 나왔다.

## 참고사이트
1. https://www.youtube.com/watch?v=B5GtZuUvujQ&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm&index=42
2. https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-12-0-rnn_basics.ipynb